# check 2012 chemoproteomic pos/protein sequence against 2018 human proteome reference

Pmap project 

11.10.19

mfpfox

# PAPER methods
- 3 sections on reasons IDs were dropped for final reference set of **3963 UKB IDs**

---

## Reasons IDs/positions dropped: 

#### 1. [A] 2012 origianl data compared to 11_2012 fasta file, IDs not found: 4119-26 = 4093

#### 2. [A] for 4093 found in both 2012 files, did position overlap with detected CK, 12 C (reactive and pantarget, no K) and 9 IDs, total 4084 IDs
    - dropID = ["P04406", "P08238", "P54136", "P60174", "P62826", "P68371", "Q4G0P3", "Q96TA1", "Q9NXV6"]

> OG positions to 11_2012 FASTA MAPPING LOST ACCOUNTING: 26 IDs not found, 9 IDs missing pos (no alignment option), TOTAL 35 IDs dropped = lost 106C & 114K

#### 3. [B] 11_2012 4084 IDs compared to 2018 fasta UKBCCDS set
    - 4084-3976 = 108 IDs not found, reasons listed in cell below
    
#### 4. [B] sequence identity comparison & MISMAP scoring for 11_2012 fasta vs 2018 fasta - 3976 IDs
    - 9 ID dropped - no position found, total 8K and 7C
    - 3 IDs dropped majority of positions not found, but 1 or 2 detected positions found
        - PRMT1 example
        - T-COFFEE alignment results downloaded!
    
```python
# shape 3976, 8, mismap scored
  identical_seq  Count
0          True   3931
1         False     45

# filtering out 12 IDs based on mismap scores, dumpme IDs had no positions found or majority of positions not found
dumpme = ["Q86YV5","Q9UKL0","P20338","Q99996","Q9Y2K2","P06753","Q66PJ3","Q99873","Q5VZ89","Q03252","Q96PV6","P49750"]

# post MISMAP score filtering
identical_seq  Count
0          True   3931
1         False     33

cleanmismap.to_csv("UKB2018_reference_dict_mismap_3964.csv", index=False)
clean2018.to_csv("UKB2018_reference_proteome_mappedCKpositions_3964.csv", index=False)
```

#### 5. [B] double check positions in FALSE identity to 11_2012 fasta are not coincidentally found
- script finds difference between two protein sequences and the pos of difference, made sure none of the positions of diff could cause accidental FINDING OF CK position.
    - lost K62 Q8WUH1, K62 is most likely not the same K detected because all surrounding positions are different
    - Churchhill protein example, not well characterized

<https://www.uniprot.org/uniprot/Q8WUH1>			
* from ukb website- CAUTION: 	It is uncertain whether Met-1 or Met-28 is the initiator. Some orthologous sequences cannot be extended. The protein analyzed in PubMed:17610897 is derived from a construct starting at Met-28

> 11_2012 to 2018 FASTA MAPPING LOST ACCOUNTING: 108 IDs not found, 12 IDs with False identity dropped for missing positions (all or majority), 1 ID dropped after checking FALSE IDENTITY proteins for accidental findings (CHURCHHILL PROTEIN EX.), total 121 IDs dropped 

# final ID count == 3963!

---
---
---
---

## input files: 
**from 2012**
- UKBID_2012reference_4084.csv
- cravattlab_11_2012.csv (needs to be filtered for IDs)

**from 2018**
- UKB_CCDS_2018_06_fasta.csv

## double checking positions in FALSE sequence match IDs
- UKBFALSEseq_doubleCheck_CK_found.csv (33 total)
- UKBseq_diff_2012_2018_ref_45diff.csv (info on what is different about sequence from 2012 fasta)
**way to confirm CK positions in non identical 2018 sequence are correct= all CK detected positions should be before the changes that caused non identical sequence match.**

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from ast import literal_eval 

In [2]:
sys.path.append("/Users/mariapalafox/Desktop/Toolbox")
from all_funx import *

In [3]:
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

In [4]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CYS_LYS_2012_to2018UKBpositions/")
print(os.listdir())

['dictOfPositions_allCysLys_4084.csv', 'REFERENCE_2018_mappedCK', 'UKBref_2018_T3931_F45.csv', '.DS_Store', 'REFERENCE_2012_accountingCKpos', 'CravattLAB', 'MAPPED_CK_v1', 'NATURE2012_UKBccds2018_files', '2012_01_UKB_Swiss_release', 'OG_to_11_2012_POS_NOT_FOUND_DROP_THESE_IDS.csv', 'all2012_datasets_not_filtered', '2013_01_UKB_Swiss_release']


# [0] filter fasta sequences from 2012 for IDs contained in 4084 list 
- (notes on 2012 datasets in part A markdown, counts in Mapping_library)

In [5]:
ids = pd.read_csv("UKBID_2012reference_4084.csv")
fasta12 = pd.read_csv("cravattlab_11_2012.csv")
print(ids.shape)
print(fasta12.shape)
print()
fasta18 = pd.read_csv("UKB_CCDS_2018_06_fasta.csv")
print(fasta18.shape)

(4084, 1)
(19981, 4)

(18432, 5)


In [10]:
# same as above but also drops False value rows
def addcolumnconditionalDrop(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:, newcol] = mendel
    df.drop(df[df[newcol] == "False"].index, inplace = True)
    df.reset_index(inplace=True, drop=True)
    print(df.shape)
    return df

In [11]:
fasta12.head(3)

,ID,entryName,Length,proSequence
0,Q8NGR1,O13A1_HUMAN,328,MKLWMESHLIVPETRPSPRMMSNQTLVTEFILQGFSEHPEYRVFLF...
1,Q8N6Q8,MET25_HUMAN,603,MAASCPLPVTPDLPTLRAKLQGLLQFLRDALSISNAHTVDFYTESV...
2,Q9UBK2,PRGC1_HUMAN,798,MAWDMCNQDSESVWSDIECAALVGEDQPLCPDLPELDLSELDVNDL...


In [9]:
dropUnnamed(fasta18)
fasta18.head(3)

,ID,entryName,Length,proSequence
0,O95139,NDUB6_HUMAN,128,MTGYTPDEKLRLQQLRELRRRWLKDQELSPREPVLPPQKMGPMEKF...
1,O75438,NDUB1_HUMAN,58,MVNLLQIVRDHWVHVLVPMGFVIGCYLDRKSDERLTAFRNKSMLFK...
2,Q8N4C6,NIN_HUMAN,2090,MDEVEQDQHEARLKELFDSFDTTGTGSLGQEELTDLCHMLSLEEVA...


In [12]:
idlist = list(ids.ID)
seq2012 = addcolumnconditionalDrop(idlist, fasta12, 'ID','in4084')
seq2018 = addcolumnconditionalDrop(idlist, fasta18, 'ID', 'in4084')

(4084, 5)
(3976, 5)


In [14]:
4084-3976

108

In [15]:
id12 = set(seq2012.ID)
id18 = set(seq2018.ID)

diff = list(id12 - id18)
print(len(diff))
print(diff)

108
['E7EVH7', 'E7EQZ4', 'O00422', 'Q9NVE7', 'Q53H96', 'P05534', 'P16455', 'O60361', 'P49189', 'P42858', 'Q9H7T3', 'P0CB43', 'Q6P4F2', 'Q5JZ73', 'P49411', 'Q9H9E3', 'A0JLT2', 'H3BQZ7', 'Q9Y4D8', 'Q6NZY4', 'H3BN98', 'P56556', 'Q7Z3J2', 'O75592', 'Q7Z460', 'J3QR44', 'P0CG05', 'P29034', 'P16435', 'F8VZW7', 'E2QRD5', 'Q99836', 'Q9NX70', 'Q8IY67', 'Q16181', 'B0V043', 'H0Y2S0', 'E7EVK2', 'P62158', 'Q9BWM7', 'P57053', 'Q6ZNA5', 'Q9H3P2', 'Q8NAG6', 'O75122', 'P09525', 'Q96I15', 'Q6DKI1', 'P62333', 'I3L499', 'Q9Y618', 'P30460', 'J3KR12', 'Q9Y5J9', 'C9J4G0', 'P01871', 'P01892', 'P0DI82', 'Q9H6E5', 'P08107', 'Q5VYK3', 'Q86YG4', 'E7ETI0', 'P30085', 'Q9Y6A9', 'H3BQ06', 'Q9Y5A7', 'P61758', 'Q8IXH7', 'P17405', 'P0CW22', 'Q8WYJ6', 'Q96EY9', 'Q8N5W9', 'P49454', 'O43504', 'Q5VTU8', 'Q86V81', 'A6NE09', 'A2A2Z9', 'Q9NYZ3', 'Q8NDA2', 'Q9H1Z4', 'D6RAD4', 'P62861', 'I3L420', 'P24928', 'P07988', 'Q8N9T8', 'H0YGG7', 'P30042', 'F5H5P2', 'Q9H3K6', 'P26440', 'Q8WX92', 'Q9Y4K4', 'P55145', 'H3BVE0', 'H7BZ11', 'A8MV

## REASONS why 108 IDs not in 2018 but in 2012 fasta: 
1. ID not apart of CCDS set, which is filtered used to make 2018 reference proteome (uncharacterized pro example)
2. ID not apart of Human Proteome set of IDs in UKB
3. ID no longer in UKB
4. ID merged with another to form new ID
5. ID was demerged to form two new IDs

# [1] identity check between 2012 sequences and 2018 sequences

In [18]:
seq2012.head(4)

,ID,entryName,Length,proSequence,in4084
0,P35998,PRS7_HUMAN,433,MPDYLGADQRKTKEDEKDDKPIRALDEGDIALLKTYGQSTYSRQIK...,True
1,Q96L92,SNX27_HUMAN,541,MADEDGEGIHPSAPHRNGGGGGGGGSGLHCAGNGGGGGGGPRVVRI...,True
2,Q2WGJ9,FR1L6_HUMAN,1857,MFGLKVKKKRNKAEKGLILANKAAKDSQGDTEALQEEPSHQEGPRG...,True
3,Q99590,SCAFB_HUMAN,1463,MKKKTVCTLNMGDKKYEDMEGEENGDNTISTGLLYSEADRCPICLN...,True


In [19]:
seq2018.head(4)

,ID,entryName,Length,proSequence,in4084
0,Q9HAS0,NJMU_HUMAN,396,MLPSLQESMDGDEKELESSEEGGSAEERRLEPPSSSHYCLYSYRGS...,True
1,Q86X76,NIT1_HUMAN,327,MLGFITRPPHRFLSLLCPGLRIPQLSVLCAQPRPRAMAISSSSCEL...,True
2,Q9NQR4,NIT2_HUMAN,276,MTSFRLALIQLQISSIKSDNVTRACSFIREAATQGAKIVSLPECFN...,True
3,Q9Y314,NOSIP_HUMAN,301,MTRHGKNCTAGAVYTYHEKKKDTAASGYGTQNIRLSRDAVKDFDCC...,True


In [37]:
# assumes ID and proSequence column
# compares the sequence in df and dict value, adds column based on comparison results
# True is they are identical, False is they are different, no info on HOW diff they are
def identicalSequenceCheck(dfref, dfalt):
    ref_dic = dict(zip(dfref.ID, dfref.proSequence))
    ref_len = dict(zip(dfref.ID, dfref.Length))
    newcol = []
    len2012 = []
    for index, row in dfalt.iterrows():
        pep = row['proSequence']
        ukb_id = row['ID']
        # retrieve uniprot fasta seq using reference dictionary
        mypep = ref_dic[ukb_id]
        len12 = ref_len[ukb_id]
        str(mypep)
        if mypep == pep:
            newcol.append('True')
            len2012.append(len12)
        if mypep != pep: 
            newcol.append('False')
            len2012.append(len12)
    # add identity score list as new column
    dfalt.loc[:,'identical_seq'] = newcol
    dfalt.loc[:,'length_2012_fasta'] = len2012
    dfalt.loc[:,'len12_minus_len18'] = dfalt['length_2012_fasta'] - dfalt['Length']
    print(dfalt.shape)
    checkColumnValues(dfalt, "identical_seq")
    return dfalt

In [39]:
checked2018 = identicalSequenceCheck(seq2012, seq2018)

(3976, 8)
  identical_seq  Count
0          True   3931
1         False     45


In [50]:
checked2018.to_csv("UKBref_2018_T3931_F45.csv", index=False)

In [40]:
false45 = checked2018[checked2018['identical_seq'] == 'False']

In [46]:
# assumes ID and proSequence column
# $$$
def sequenceDifference(dfref, dfalt):
    ref_dic = dict(zip(dfref.ID, dfref.proSequence))
    diffcol = []
    for index, row in dfalt.iterrows():
        pep = row['proSequence']
        ukb_id = row['ID']
        # retrieve uniprot fasta seq using reference dictionary
        mypep = ref_dic[ukb_id]
        str(mypep)
        if mypep == pep:
            diffcol.append('nodiff')
        if mypep != pep: 
            output_list = [li for li in difflib.ndiff(mypep, pep) if li[0] != ' ']
            out = ', '.join(output_list)
            diffcol.append(out)
    # add identity score list as new column
    dfalt.loc[:,'difference_12vs18'] = diffcol
    print(dfalt.shape)
    return dfalt

In [47]:
diff2018 = sequenceDifference(seq2012, false45)

(45, 9)


/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [48]:
diff2018

,ID,entryName,Length,proSequence,in4084,identical_seq,length_2012_fasta,len12_minus_len18,difference_12vs18
24,P14598,NCF1_HUMAN,390,MGDTFIRHIALLGFEKRFVPSQHYVYMFLVKWQDLSEKVVYRRFTE...,True,False,390,0,"- G, + S"
261,Q96CV9,OPTN_HUMAN,577,MSHQPLSCLTEKEDSPSESTGNGPPHLAHPNLDTFTPEELLQQMKE...,True,False,577,0,"- K, + E"
328,Q86YV5,PRAG1_HUMAN,1406,MHQTLCLNPESLKMSACSDFVEHIWKPGSCKNCFCLRSDHQLVAGP...,True,False,1402,-4,"+ S, + G, + P, + A"
333,Q9H307,PININ_HUMAN,717,MAVAVRTLQEQLEKAKESLKNVDENIRKLTGRDPNDVRPIQARLLA...,True,False,717,0,"- S, - L, - S, - P, - G, - K, - E, - N, - V, -..."
343,Q9NPH0,PPA6_HUMAN,428,MITGVFSMRLWTPVGVLTSLAYCLHQRRVALAELQEADGQCPVDRS...,True,False,428,0,"- V, + M"
414,Q15283,RASA2_HUMAN,850,MAAAAPAAAAASSEAPAASATAEPEAGDQDSREVRVLQSLRGKICE...,True,False,849,-1,+ S
423,Q9UKL0,RCOR1_HUMAN,485,MPAMVEKGPEVSGKRRGRNNAAASASAAAASAAASAACASPAATAA...,True,False,482,-3,"+ M, + P, + A"
439,P20338,RAB4A_HUMAN,218,MSQTAMSETYDFLFKFLVIGNAGTGKSCLLHQFIEKKFKDDSNHTI...,True,False,213,-5,"+ M, + S, + Q, + T, + A"
551,Q6WKZ4,RFIP1_HUMAN,1283,MSLMVSAGRGLGAVWSPTHVQVTVLQARGLRAKGPGGTSDAYAVIQ...,True,False,1283,0,"- K, - A, - K, - S, - E, - G, - P, - P, - L, -..."
563,Q16822,PCKGM_HUMAN,640,MAALYRPGLRLNWHGLSPLGWPSCRSIQTLRVLSGDLGQLPTGIRD...,True,False,640,0,"- Q, - L, - P, - G, - G, - A, - R, - G, - Q, -..."


In [49]:
diff2018.to_csv("UKBseq_diff_2012_2018_ref_45diff.csv", index=False)

# [2] position of detected C & K check 

In [54]:
# FROM Pmap_make_pos_dict.py

# funx that turns list of labeled positions into sorted dictionary
# each key is index+1 of cys or lys that was labeled
# each value is chr of C or K
def get_pos_dictionary(pdcol):
    list_of_dicts = []
    for row in pdcol.tolist():
        listofkey=[]
        listofvalues=[]
        for i in row:
            v = i[0]
            listofvalues.append(v)
            k = i[1:]
            listofkey.append(k)
        di = dict(zip(listofkey, listofvalues))
        intdict = {int(oldkey): val for oldkey, val in di.items()}
        sdi = dict(sorted(intdict.items(), reverse=False))
        list_of_dicts.append(sdi)
    return list_of_dicts
def make_pos_dict():
    # import files
    cysever = "Cys2012_ever_labeled_6404.csv"
    lysever = "Lys2012_ever_labeled_9213.csv"
    cysever = pd.read_csv(cysever)
    lysever = pd.read_csv(lysever)
    # drop columns that i dont need
    cys = cysever.filter(['entry', 'pos'], axis=1)
    lys = lysever.filter(['entry', 'pos'], axis=1)
    # appending together at bottom, total shape is 
    appendedcyslys = cys.append(lys, ignore_index=True)
    print("appendedcyslys shape: ", appendedcyslys.shape)
    # creating list of positions labeled for each entry
    groupedCYSLYS = appendedcyslys.groupby('entry')['pos'].apply(list)
    groupedCYSLYS2 = pd.DataFrame(groupedCYSLYS, index=None)
    groupedCYSLYS2.reset_index(inplace=True)
    # creating column that is # of times entry has been labeled
    groupedCYSLYS2['labeled_pos_count'] = groupedCYSLYS2['pos'].str.len()
    # sorting df by decending # of times entry has been labeled
    groupedCYSLYS2_sort = groupedCYSLYS2.sort_values('labeled_pos_count',ascending=False)
    groupedCYSLYS2_sort.reset_index(inplace=True, drop=True)
    # calling funx by passing in df series (pos column)
    list_of_dicts = get_pos_dictionary(groupedCYSLYS2_sort['pos'])
    # turning list of dictionaries into a pandas column
    dictSeries = pd.Series(list_of_dicts)
    # adding dict col to original dataframe
    groupedCYSLYS2_sort['pos_dict'] = dictSeries
    # saving new file with positions of labeled cys and lys in dictionary form
    groupedCYSLYS2_sort.to_csv("dictOfPositions_allCysLys_.csv")
    return groupedCYSLYS2_sort

In [55]:
posdic12 = make_pos_dict()

appendedcyslys shape:  (15617, 2)


In [62]:
posdic12.columns = ['ID', 'pos', 'labeled_pos_count', 'pos_dict']
posdic12.head(3)

,ID,pos,labeled_pos_count,pos_dict
0,Q09666,"[C108, C1833, C1900, C1967, C2162, C2806, C550...",112,"{85: 'K', 108: 'C', 128: 'K', 243: 'K', 252: '..."
1,P21333,"[C1018, C1157, C1165, C1260, C1353, C1453, C17...",75,"{53: 'C', 58: 'K', 87: 'K', 210: 'C', 220: 'K'..."
2,O75369,"[C1087, C1158, C1280, C1326, C1617, C183, C186...",65,"{15: 'K', 16: 'K', 26: 'C', 60: 'K', 93: 'K', ..."


In [61]:
checked2018.head(3)

,ID,entryName,Length,proSequence,in4084,identical_seq,length_2012_fasta,len12_minus_len18
0,Q9HAS0,NJMU_HUMAN,396,MLPSLQESMDGDEKELESSEEGGSAEERRLEPPSSSHYCLYSYRGS...,True,True,396,0
1,Q86X76,NIT1_HUMAN,327,MLGFITRPPHRFLSLLCPGLRIPQLSVLCAQPRPRAMAISSSSCEL...,True,True,327,0
2,Q9NQR4,NIT2_HUMAN,276,MTSFRLALIQLQISSIKSDNVTRACSFIREAATQGAKIVSLPECFN...,True,True,276,0


In [78]:
# df has ID col and protein sequences
# pos df has ID col and dictionary of positions
# merge col is shared col between two files
from ast import literal_eval # for mismap_score func
def mismap_score(df, posdf, mergecol):
    score_file = []
    # merge two df
    mer = pd.merge(df, posdf, how='inner', on=[mergecol])
    # QC merge
    print('shape ORIGINAL  df: ', df.shape)
    print('shape merge: ',mer.shape)
    print('shape ORIGINAL KEY ref: ', posdf.shape)
    print()
    for index, row in mer.iterrows():
        newrow = []
        ukb_id = row['ID']
        #ensp = row['ENSP']
        #ensp_len = row['Length']
        #ukb_len = row['UniProt_length']
        #C_abun = row['C_abundance']
        #K_abun = row['K_abundance']
        identical = row['identical_seq']
        len18 = row['Length']
        tot_tar = row['labeled_pos_count']
        pep = row['proSequence']
        posCK = row['pos_dict']


        # col order
        # newrow.append(ensp)
        newrow.append(ukb_id)
        newrow.append(identical)
        newrow.append(len18)
        #newrow.append(ukb_len)
        #newrow.append(ensp_len)
        #newrow.append(C_abun)
        #newrow.append(K_abun)
        newrow.append(tot_tar)

        # positions found counter
        count = 0
        foundC = 0
        foundK = 0
        missedC = 0
        missedK = 0

        # evaluate as dictionary
        #python_dict = literal_eval(posCK) 
        python_dict = posCK # already was dict object
        resultfound = []
        resultnotfound = []
        
        # loop thru position dict
        for key in python_dict:
            k = int(key)
            # convert pos # to index 0-based
            i = k - 1
            # if index within ensembl/or ukb protein string
            if i < int(len18):
                # get AA from peptide string
                AA = pep[i]
                # get AA from pos dic
                checker = python_dict[key]
                posid = checker + str(k)
                # match, increase count
                if AA == checker:
                    count += 1
                    resultfound.append(posid)
                    if AA == 'C':
                        foundC += 1
                    if AA == 'K':
                        foundK += 1
                if AA != checker:
                    resultnotfound.append(posid)
                    if AA == 'C':
                        missedC += 1
                    if AA == 'K':
                        missedK += 1

        # fraction correct pos
        cor_per = count/tot_tar
        miss = tot_tar - count
        # fraction missed
        miss_per = miss/tot_tar

        # add columns
        newrow.append(count) # found count
        newrow.append(miss) # missed count
        newrow.append(round(cor_per, 2))
        newrow.append(round(miss_per, 2))
        resfound = ', '.join(resultfound)
        resnotfound = ', '.join(resultnotfound)
        newrow.append(resfound)
        newrow.append(resnotfound)
        newrow.append(posCK)
        newrow.append(foundC)
        newrow.append(missedC)
        newrow.append(foundK)
        newrow.append(missedK)
        score_file.append(newrow)

    # turn list of list into dataframe
    df = pd.DataFrame(score_file)
    df.columns = ['UniProtID', 'identical_seq','len_uniprot18','total_targets','found_count','missed_count','correct_frac','missed_frac', 'positions_found', 'positions_notfound','pos_dict','found_C_count','missed_C_count','found_K_count','missed_K_count']
    #df.columns = ['UniProtID', 'identical_seq','len_uniprot18','len_ensp', 'C_abun_ukb','K_abun_ukb','total_targets','found_count','missed_count','correct_frac','missed_frac']
    return df

In [79]:
mismapdf = mismap_score(checked2018, posdic12, 'ID')

shape ORIGINAL  df:  (3976, 8)
shape merge:  (3976, 11)
shape ORIGINAL KEY ref:  (4084, 4)



In [80]:
mismapdf[mismapdf['identical_seq']== 'False']

,UniProtID,identical_seq,len_uniprot18,total_targets,found_count,missed_count,correct_frac,missed_frac,positions_found,positions_notfound,pos_dict,found_C_count,missed_C_count,found_K_count,missed_K_count
24,P14598,False,390,2,2,0,1.00,0.00,"C111, C378",,"{111: 'C', 378: 'C'}",2,0,0,0
261,Q96CV9,False,577,2,2,0,1.00,0.00,"K106, C239",,"{106: 'K', 239: 'C'}",1,0,1,0
328,Q86YV5,False,1406,3,2,1,0.67,0.33,"C155, C221",C444,"{155: 'C', 221: 'C', 444: 'C'}",2,0,0,0
333,Q9H307,False,717,2,2,0,1.00,0.00,"C249, C439",,"{249: 'C', 439: 'C'}",2,0,0,0
343,Q9NPH0,False,428,2,2,0,1.00,0.00,"C267, K291",,"{267: 'C', 291: 'K'}",1,0,1,0
414,Q15283,False,850,1,1,0,1.00,0.00,C314,,{314: 'C'},1,0,0,0
423,Q9UKL0,False,485,2,0,2,0.00,1.00,,"C35, K418","{35: 'C', 418: 'K'}",0,0,0,0
439,P20338,False,218,1,0,1,0.00,1.00,,K58,{58: 'K'},0,0,0,0
551,Q6WKZ4,False,1283,1,1,0,1.00,0.00,C318,,{318: 'C'},1,0,0,0
563,Q16822,False,640,4,4,0,1.00,0.00,"C92, C210, C306, C431",,"{92: 'C', 210: 'C', 306: 'C', 431: 'C'}",4,0,0,0


In [81]:
checkColumnValues(mismapdf, 'identical_seq')

  identical_seq  Count
0          True   3931
1         False     45


In [83]:
mismapdf.to_csv("UKB_2018_mismap_scored_sequences_3976.csv", index=False)

# Reason for loss: 12 IDs dropped because 
1. no target position was found when searching 2018 seq. 
2. target was found but a majority of other targets in ID not found, higher likelihood that position found is coincidence if majority others not found

```
UniProtID	identical_seq	len_uniprot18	total_targets	found_count	missed_count	correct_frac	missed_frac	positions_found	positions_notfound	pos_dict	found_C_count	missed_C_count	found_K_count	missed_K_count
Q9UKL0	FALSE	485	2	0	2	0	1		C35, K418	{35: 'C', 418: 'K'}	0	0	0	0
P20338	FALSE	218	1	0	1	0	1		K58	{58: 'K'}	0	0	0	0
Q99996	FALSE	3907	2	0	2	0	1		C1966, C3868	{1966: 'C', 3868: 'C'}	0	0	0	0
Q9Y2K2	FALSE	1321	2	0	2	0	1		K23, K31	{23: 'K', 31: 'K'}	0	0	0	0
Q66PJ3	FALSE	421	2	0	2	0	1		C159, C349	{159: 'C', 349: 'C'}	0	0	0	0
Q5VZ89	FALSE	1909	2	0	2	0	1		K30, C850	{30: 'K', 850: 'C'}	0	0	0	0
Q03252	FALSE	620	2	0	2	0	1		K175, K500	{175: 'K', 500: 'K'}	0	0	0	0
Q96PV6	FALSE	800	1	0	1	0	1		C529	{529: 'C'}	0	0	0	0
P49750	FALSE	2146	1	0	1	0	1		K1657	{1657: 'K'}	0	0	0	0
Q99873	FALSE	371	13	1	12	0.08	0.92	K233	C93, K98, K103, C109, K124, K135, C216, C262, K266, K303, K332, C350	{93: 'C', 98: 'K', 103: 'K', 109: 'C', 124: 'K', 135: 'K', 216: 'C', 233: 'K', 262: 'C', 266: 'K', 303: 'K', 332: 'K', 350: 'C'}	0	1	1	0
P06753	FALSE	285	7	1	6	0.14	0.86	K7	K136, K149, K168, K217, K231, K248	{7: 'K', 136: 'K', 149: 'K', 168: 'K', 217: 'K', 231: 'K', 248: 'K'}	0	0	1	0
Q86YV5	FALSE	1406	3	2	1	0.67	0.33	C155, C221	C444	{155: 'C', 221: 'C', 444: 'C'}	2	0	0	0!
```

In [93]:
# IDs that had no positions found or majority of positions not found
dumpme = ["Q86YV5","Q9UKL0","P20338","Q99996","Q9Y2K2","P06753","Q66PJ3","Q99873","Q5VZ89","Q03252","Q96PV6","P49750"]

In [94]:
def addcolumnconditionalDropTrue(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:, newcol] = mendel
    df.drop(df[df[newcol] == "True"].index, inplace=True)
    df.reset_index(inplace=True, drop=True)
    print("new shape post drop: ", df.shape)
    return df

In [95]:
clean2018 = addcolumnconditionalDropTrue(dumpme, checked2018, 'ID', 'missed_majority_pos')

new shape post drop:  (3964, 9)


In [96]:
cleanmismap = addcolumnconditionalDropTrue(dumpme, mismapdf, 'UniProtID', 'missed_majority_pos')

new shape post drop:  (3964, 16)


In [97]:
checkColumnValues(clean2018, 'identical_seq')

  identical_seq  Count
0          True   3931
1         False     33


In [98]:
checkColumnValues(cleanmismap, 'identical_seq')

  identical_seq  Count
0          True   3931
1         False     33


In [100]:
cleanmismap.to_csv("UKB2018_reference_dict_mismap_3964.csv", index=False)
clean2018.to_csv("UKB2018_reference_proteome_mappedCKpositions_3964.csv", index=False)

# [3] using reference 3964 IDs with CK positions mapped to 2018, clean subset files for accurate 2018 accounting

In [102]:
idlist = list(cleanmismap.UniProtID)
print(len(idlist))
print(len(set(idlist)))

3964
3964


In [104]:
# same as above but also drops False value rows
def addcolumnconditionalDrop(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:, newcol] = mendel
    df.drop(df[df[newcol] == "False"].index, inplace = True)
    df.reset_index(inplace=True, drop=True)
    print(df.shape)
    return df

In [106]:
#df2012 = ["AllReact2012_5959.csv","ALLReact2012_grouped_entry_threshold_2807.csv","Cys2012_ever_labeled_6404.csv","CYS2012_everlabeled_gene_valueCounts_3006.csv","Cys2012_pantarget_6125.csv","Cys2012_reactive_1458.csv","CYSLYS2012_everlabeled_gene_valueCounts_4084.csv","dictOfPositions_allCysLys_4084.csv","Lys2012_ever_labeled_9213.csv","LYS2012_everlabeled_gene_valueCounts_2683.csv","Lys2012_pantarget_8257.csv"]
#df2018 = ["AllReact2018_5959.csv","ALLReact2018_grouped_entry_threshold_2807.csv","Cys2018_ever_labeled_6404.csv","CYS2018_everlabeled_gene_valueCounts_3006.csv","Cys2018_pantarget_6125.csv","Cys2018_reactive_1458.csv","CYSLYS2018_everlabeled_gene_valueCounts_4084.csv","dictOfPositions_allCysLys_4084.csv","Lys2018_ever_labeled_9213.csv","LYS2018_everlabeled_gene_valueCounts_2683.csv","Lys2018_pantarget_8257.csv"]
df2012 = ["Lys2012_reactive_4501.csv"]
df2018 = ["Lys2018_reactive_4501.csv"]
for i,d in enumerate(df2012): 
    df = pd.read_csv(d)
    print(d)
    df2 = addcolumnconditionalDrop(idlist, df, 'ID','final_refID')
    outname = df2018[i]
    df2.to_csv(outname)

Lys2012_reactive_4501.csv
(4425, 7)


## all results after filtering for 3964 IDs in 2018 with mapped positions
```bash
AllReact2012_5959.csv
(5856, 6)
ALLReact2012_grouped_entry_threshold_2807.csv
(2751, 4)
Cys2012_ever_labeled_6404.csv
(6234, 6)
CYS2012_everlabeled_gene_valueCounts_3006.csv
(2915, 3)
Cys2012_pantarget_6125.csv
(5962, 6)
Cys2012_reactive_1458.csv
(1431, 7)
CYSLYS2012_everlabeled_gene_valueCounts_4084.csv
(3964, 3)
dictOfPositions_allCysLys_4084.csv
(3964, 5)
Lys2012_ever_labeled_9213.csv
(9046, 6)
LYS2012_everlabeled_gene_valueCounts_2683.csv
(2624, 3)
Lys2012_pantarget_8257.csv
(8115, 6)
Lys2012_reactive_4501.csv
(4425, 7)
```

In [107]:
# GET ID counts in subset files

path = 'REFERENCE_2018_mappedCK/'
filelist = ["Cys2018_pantarget_5962.csv","Cys2018_reactive_1431.csv","Lys2018_pantarget_8115.csv","Lys2018_reactive_4425.csv"]

for i in filelist:
    filename = path + i
    df = pd.read_csv(filename)
    idcol = list(df.ID)
    idset = set(idcol)
    print("list lenght: ", len(idcol))
    print("set length: ", len(idset))


list lenght:  5962
set length:  2788
list lenght:  1431
set length:  984
list lenght:  8115
set length:  2370
list lenght:  4425
set length:  1545


# ACCOUNTING UPDATE with 2018 protein sequences

In [108]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CYS_LYS_2012_to2018UKBpositions/REFERENCE_2018_mappedCK/")

In [109]:
allreact = pd.read_csv("AllReact2018_5856.csv")
cpan = pd.read_csv("Cys2018_pantarget_5962.csv")
Cfinalreact = pd.read_csv("Cys2018_reactive_1431.csv")
kpan = pd.read_csv("Lys2018_pantarget_8115.csv")
Kfinalreact = pd.read_csv("Lys2018_reactive_4425.csv")

In [110]:
from collections import Counter
def colvalues(df, col):
    countdf = Counter(list(df[col]))
    countdic = dict(countdf)
    print(countdic)

In [111]:
colvalues(allreact, 'Threshold')
print()
colvalues(Cfinalreact, 'Threshold')
print()
colvalues(Kfinalreact, 'Threshold')
print()

{'High': 400, 'Medium': 1088, 'Low': 4368}

{'High': 97, 'Medium': 422, 'Low': 912}

{'High': 303, 'Medium': 666, 'Low': 3456}



In [112]:
checkColumnValues(cpan, "labelType")

     labelType  Count
0  panReactive   5228
1       Target    734


In [113]:
checkColumnValues(kpan,"labelType")

     labelType  Count
0  panReactive   7997
1       Target    118


In [116]:
dropUnnamed(Cfinalreact)


,pos_ID,reactivity,Threshold,subset,ID,pos,final_refID
0,O14733_C202,0.74,High,Cysteine,O14733,C202,True
1,P07237_C56,0.80,High,Cysteine,P07237,C56,True
2,P78417_C32,0.84,High,Cysteine,P78417,C32,True
3,Q6DCA0_C153,0.88,High,Cysteine,Q6DCA0,C153,True
4,Q9Y4X0_C175,0.88,High,Cysteine,Q9Y4X0,C175,True
5,Q00796_C45,0.89,High,Cysteine,Q00796,C45,True
6,Q9UHQ1_C99,0.89,High,Cysteine,Q9UHQ1,C99,True
7,Q9Y6I4_C302,0.90,High,Cysteine,Q9Y6I4,C302,True
8,P38606_C138,0.90,High,Cysteine,P38606,C138,True
9,Q8IY18_C91,0.92,High,Cysteine,Q8IY18,C91,True


In [117]:
dropUnnamed(Kfinalreact)

In [118]:
clow=Cfinalreact[Cfinalreact["Threshold"] == "Low"].describe()
clow.columns=['C_LOW']
cmed = Cfinalreact[Cfinalreact["Threshold"] == "Medium"].describe()
cmed.columns=['C_MED']
chigh=Cfinalreact[Cfinalreact["Threshold"] == "High"].describe()
chigh.columns=['C_HIGH']
CLM = clow.join(cmed)
CLMH = CLM.join(chigh)
print(CLMH)
CLMH.to_csv("CLMH_description_table_2018.csv")


klow = Kfinalreact[Kfinalreact["Threshold"] == "Low"].describe()
klow.columns = ['K_LOW']
kmed = Kfinalreact[Kfinalreact["Threshold"] == "Medium"].describe()
kmed.columns = ['K_MED']
khigh=Kfinalreact[Kfinalreact["Threshold"] == "High"].describe()
khigh.columns=['K_HIGH']
klm = klow.join(kmed)
klmh = klm.join(khigh)
print(klmh)
klmh.to_csv("KLMH_description_table_2018.csv")

            C_LOW       C_MED     C_HIGH
count  912.000000  422.000000  97.000000
mean     7.417007    3.851872   1.336804
std      3.056919    0.831851   0.340152
min      5.010000    2.010000   0.740000
25%      5.737500    3.192500   1.060000
50%      6.460000    4.015000   1.320000
75%      7.870000    4.572500   1.640000
max     20.000000    5.000000   1.960000
             K_LOW       K_MED      K_HIGH
count  3456.000000  666.000000  303.000000
mean      9.334259    3.518769    1.240264
std       1.341434    0.881246    0.494368
min       5.100000    2.100000    0.100000
25%       9.900000    2.700000    0.900000
50%      10.000000    3.500000    1.300000
75%      10.000000    4.300000    1.700000
max      10.000000    5.000000    2.000000


In [120]:
dropUnnamed(allreact)

In [121]:
allreact.describe()

,reactivity
count,5856.000000
mean,7.427932
std,3.173233
min,0.100000
25%,5.000000
50%,8.700000
75%,10.000000
max,20.000000


In [122]:
Cfinalreact.describe()

,reactivity
count,1431.000000
mean,5.953508
std,3.206224
min,0.740000
25%,4.320000
50%,5.620000
75%,6.850000
max,20.000000


In [123]:
Kfinalreact.describe()

,reactivity
count,4425.000000
mean,7.904746
std,3.012153
min,0.100000
25%,5.800000
50%,10.000000
75%,10.000000
max,10.000000


---
---
---

# Double checking CK pos from 2018 sequences that != 2012 sequences

In [5]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CYS_LYS_2012_to2018UKBpositions/REFERENCE_2018_mappedCK/")
print(os.listdir())

['UKBID_2012reference_4084.csv', 'CCDS2018_all_Cpos.csv', '.DS_Store', 'tcoffee-CHURHILL_all-files-aa5cecb1.zip', 'UKB_DOUBLE_CHECK_CKpos_forFALSEidentical_sequences_33.xlsx', 'FILTERING', 'NOT_FOUND_in2018_sequenceUKB_dropIDs.xlsx', 'tcoffee-PRMT1_all-files-4cc6ed53.zip', 'UKB_DOUBLE_CHECK_CKpos_forFALSEidentical_sequences_33.csv', 'CCDS2018_all_Kpos.csv', 'UKB2018_reference_dict_mismap_3963.xlsx', 'UKB_CCDS_2018_06_fasta.csv', 'FALSE45_11_2012vs2018fasta.xlsx', 'tcoffee-CHURHILL_all-files-aa5cecb1', 'UKB2018_reference_dict_mismap_3963.csv', 'SUBSETS', 'cravattlab_11_2012.csv', 'tcoffee-PRMT1_all-files-4cc6ed53']


## [0] ULTIMATE FASTA SEQ COMPARISON SCRIPT BELOW!!! 
**adding difference in protein length between 2012 and 2018 to final set of 3964 CK positions file before checking False**
- organized code from this markdown in to Pmap_protein_sequence_comparison.py
- only adds diff in AA and position for subset of non identicacl protein seqs
- ID list made from already filtered 3964 IDs

WHY ndiff was outputting strange elements- https://stackoverflow.com/questions/54960450/why-do-i-get-a-stray-element-with-difflib-ndiff

**using simple diff in string finder instead of ndiff to avoid weird errors**

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# 11/14/19 Pmap_protein_sequence_comparison.py

"""
- input files must have  ID and proSequence column (fasta -> CSV output files)
- compares the prosequence in df against ID:seqeunce dict created, adds column True for identical False for not identical
- seqeunceDifference funx takes only False identity rows and identifies WHAT is different about them
    - added code for positions (1 based) that are different
- made from MARKDOWN 'B'
"""

import os
import sys
import pandas as pd
from ast import literal_eval
import difflib

sys.path.append("/Users/mariapalafox/Desktop/Toolbox")
from all_funx import *



def identicalSequenceCheck(dfref, dfalt):
    ref_dic = dict(zip(dfref.ID, dfref.proSequence))
    ref_len = dict(zip(dfref.ID, dfref.Length))
    newcol = []
    len2012 = []
    for index, row in dfalt.iterrows():
        pep = row['proSequence']
        ukb_id = row['ID']
        # retrieve uniprot fasta seq using reference dictionary
        mypep = ref_dic[ukb_id]
        len12 = ref_len[ukb_id]
        str(mypep)
        str(pep)
        if mypep == pep:
            newcol.append('True')
            len2012.append(len12)
        if mypep != pep: 
            newcol.append('False')
            len2012.append(len12)
    # add identity score list as new column
    dfalt.loc[:,'identical_seq'] = newcol
    dfalt.loc[:,'length_2012_fasta'] = len2012
    dfalt.loc[:,'len12_minus_len18'] = dfalt['length_2012_fasta'] - dfalt['Length']
    print(dfalt.shape)
    checkColumnValues(dfalt, "identical_seq")
    return dfalt

# assumes ID and proSequence column
# $$$
# dfref is all 2012 seqeucne dfalt is only non identical sequences 2018
def sequenceDifference(dfref, dfalt):
    ref_dic = dict(zip(dfref.ID, dfref.proSequence))
    diffcol = []
    posdiffcol = []
    for index, row in dfalt.iterrows():
        altpep = row['proSequence']
        ukb_id = row['ID']
        # retrieve 2012seq from dict
        refpep = ref_dic[ukb_id]
        str(refpep)
        str(altpep)
        lenref = len(refpep)
        lenalt = len(altpep)

        # len check is for posout_list2, shorter pep must be in range(len)
        if lenref == lenalt:
            output_list = [altpep[i] for i in range(len(altpep)) if altpep[i] != refpep[i]]
            posoutput_list2 = [i+1 for i in range(len(altpep)) if altpep[i] != refpep[i]]
            out = ', '.join(output_list)
            #posout = ', '.join(posoutput_list2)
            diffcol.append(out)
            posdiffcol.append(posoutput_list2)

        # 2012 protein longer than 2018
        if lenref > lenalt:
            output_list = [altpep[i] for i in range(len(altpep)) if altpep[i] != refpep[i]]
            posoutput_list2 = [i+1 for i in range(len(altpep)) if altpep[i] != refpep[i]]
            out = ', '.join(output_list)
            #posout = ', '.join(posoutput_list2)
            diffcol.append(out)
            posdiffcol.append(posoutput_list2)

        # 2018 protein longer than 2012
        if lenref < lenalt:
            output_list = [altpep[i] for i in range(len(refpep)) if refpep[i] != altpep[i]]
            posoutput_list2 = [i+1 for i in range(len(refpep)) if refpep[i] != altpep[i]]
            out = ', '.join(output_list)
            #posout = ', '.join(posoutput_list2)
            diffcol.append(out)
            posdiffcol.append(posoutput_list2)

    # add diff as new column
    dfalt.loc[:, 'difference_18vs12'] = diffcol
    # add pos diff as new column
    dfalt.loc[:, 'posdiff_ref_shorter_pro'] = posdiffcol
    print("new dfalt shape with added columns: ", dfalt.shape)
    return dfalt

In [7]:
# main function for code above

# file with ID col for IDs you want to only compare sequences of
ids = pd.read_csv("UKBID_2012reference_4084.csv")
fasta12 = pd.read_csv("cravattlab_11_2012.csv") # what paper says is ref
fasta18 = pd.read_csv("UKB_CCDS_2018_06_fasta.csv")
print("ID file shape: ", ids.shape)
print("fasta 2012 shape: ", fasta12.shape)
print("fasta 2018 shape: ", fasta18.shape)
print()

# only keep IDs/sequences from both fastas that match an ID in IDlist
idlist = list(ids.ID)
seq2012 = addcolumnconditionalDrop(idlist, fasta12, 'ID', 'IDin4084list')
seq2018 = addcolumnconditionalDrop(idlist, fasta18, 'ID', 'IDin4084list')

# call funx that compares sequences
checked2018 = identicalSequenceCheck(seq2012, seq2018)
# saving all 2018 sequences in ID list with TRUE FALSE seq comparison col
checked2018.to_csv("UKBref2.0_2018_T3931_F45.csv", index=False)

# isolate FALSE sequence identity rows
falseonly = checked2018[checked2018['identical_seq'] == 'False']

# $$$ outputs what exactly is different about seqeunces
diff2018 = sequenceDifference(seq2012, falseonly)
diff2018.to_csv("UKBseq2.0_diff_45nonidentical_proSeq_2012vs2018seq.csv", index=False)


ID file shape:  (4084, 1)
fasta 2012 shape:  (19981, 4)
fasta 2018 shape:  (18432, 5)

  IDin4084list  Count
0        False  15897
1         True   4084
dropping false
df shape post drop:  (4084, 5)
  IDin4084list  Count
0        False  14456
1         True   3976
dropping false
df shape post drop:  (3976, 6)
(3976, 9)
  identical_seq  Count
0          True   3931
1         False     45
new dfalt shape with added columns:  (45, 11)


/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
# checking newly made file with 33 non identical protein sequences
# all detected CK positions should come after the change positions
diff45 = pd.read_csv("UKBseq2.0_diff_45nonidentical_proSeq_2012vs2018seq.csv")
dropUnnamed(diff45)

In [11]:
diff45

,ID,entryName,Length,proSequence,IDin4084list,identical_seq,length_2012_fasta,len12_minus_len18,difference_18vs12,posdiff_ref_shorter_pro
0,P14598,NCF1_HUMAN,390,MGDTFIRHIALLGFEKRFVPSQHYVYMFLVKWQDLSEKVVYRRFTE...,True,False,390,0,S,[99]
1,Q96CV9,OPTN_HUMAN,577,MSHQPLSCLTEKEDSPSESTGNGPPHLAHPNLDTFTPEELLQQMKE...,True,False,577,0,E,[322]
2,Q86YV5,PRAG1_HUMAN,1406,MHQTLCLNPESLKMSACSDFVEHIWKPGSCKNCFCLRSDHQLVAGP...,True,False,1402,-4,"S, G, A, S, S, P, F, V, P, H, L, E, S, D, Y, C...","[351, 352, 353, 354, 355, 356, 357, 358, 359, ..."
3,Q9H307,PININ_HUMAN,717,MAVAVRTLQEQLEKAKESLKNVDENIRKLTGRDPNDVRPIQARLLA...,True,False,717,0,T,[441]
4,Q9NPH0,PPA6_HUMAN,428,MITGVFSMRLWTPVGVLTSLAYCLHQRRVALAELQEADGQCPVDRS...,True,False,428,0,M,[316]
5,Q15283,RASA2_HUMAN,850,MAAAAPAAAAASSEAPAASATAEPEAGDQDSREVRVLQSLRGKICE...,True,False,849,-1,"S, K, D, A, I, Y, T, I, P, V, K, N, I, L, A, V...","[646, 647, 648, 649, 650, 651, 652, 653, 654, ..."
6,Q9UKL0,RCOR1_HUMAN,485,MPAMVEKGPEVSGKRRGRNNAAASASAAAASAAASAACASPAATAA...,True,False,482,-3,"P, A, M, V, E, K, G, P, E, V, S, G, K, R, G, R...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
7,P20338,RAB4A_HUMAN,218,MSQTAMSETYDFLFKFLVIGNAGTGKSCLLHQFIEKKFKDDSNHTI...,True,False,213,-5,"Q, A, M, S, E, T, Y, D, F, L, F, K, F, L, V, I...","[3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,..."
8,Q6WKZ4,RFIP1_HUMAN,1283,MSLMVSAGRGLGAVWSPTHVQVTVLQARGLRAKGPGGTSDAYAVIQ...,True,False,1283,0,"Q, A","[622, 651]"
9,Q16822,PCKGM_HUMAN,640,MAALYRPGLRLNWHGLSPLGWPSCRSIQTLRVLSGDLGQLPTGIRD...,True,False,640,0,P,[121]


In [12]:
# file with 33 CK positions to be checked 
falseCK = pd.read_csv("UKB_2018_mismap_scored_sequences_3976.csv")


In [13]:
falseCK

,UniProtID,identical_seq,len_uniprot18,total_targets,found_count,missed_count,correct_frac,missed_frac,positions_found,positions_notfound,pos_dict,found_C_count,missed_C_count,found_K_count,missed_K_count
0,Q9HAS0,True,396,1,1,0,1.0,0.0,C182,NaN,{182: 'C'},1,0,0,0
1,Q86X76,True,327,3,3,0,1.0,0.0,"K161, C165, C203",NaN,"{161: 'K', 165: 'C', 203: 'C'}",2,0,1,0
2,Q9NQR4,True,276,7,7,0,1.0,0.0,"C44, K52, K123, K130, C146, C153, C170",NaN,"{44: 'C', 52: 'K', 123: 'K', 130: 'K', 146: 'C...",4,0,3,0
3,Q9Y314,True,301,3,3,0,1.0,0.0,"C8, K78, C185",NaN,"{8: 'C', 78: 'K', 185: 'C'}",2,0,1,0
4,Q04721,True,2471,1,1,0,1.0,0.0,C2085,NaN,{2085: 'C'},1,0,0,0
5,Q9UHY1,True,535,1,1,0,1.0,0.0,C478,NaN,{478: 'C'},1,0,0,0
6,P49116,True,596,1,1,0,1.0,0.0,C387,NaN,{387: 'C'},1,0,0,0
7,Q16656,True,503,1,1,0,1.0,0.0,K145,NaN,{145: 'K'},0,0,1,0
8,Q96IY1,True,281,3,3,0,1.0,0.0,"K149, C175, C224",NaN,"{149: 'K', 175: 'C', 224: 'C'}",2,0,1,0
9,Q86UY8,True,548,2,2,0,1.0,0.0,"K260, C276",NaN,"{260: 'K', 276: 'C'}",1,0,1,0


In [14]:
falseCK.columns = ['ID', 'identical_seq', 'len_uniprot18', 'total_targets',
       'found_count', 'missed_count', 'correct_frac', 'missed_frac',
       'positions_found', 'positions_notfound', 'pos_dict','found_C_count','missed_C_count','found_K_count','missed_K_count']

In [15]:
mer = pd.merge(diff45,falseCK,how='inner',on=['ID'])

In [16]:
mer

,ID,entryName,Length,proSequence,IDin4084list,identical_seq_x,length_2012_fasta,len12_minus_len18,difference_18vs12,posdiff_ref_shorter_pro,identical_seq_y,len_uniprot18,total_targets,found_count,missed_count,correct_frac,missed_frac,positions_found,positions_notfound,pos_dict,found_C_count,missed_C_count,found_K_count,missed_K_count
0,P14598,NCF1_HUMAN,390,MGDTFIRHIALLGFEKRFVPSQHYVYMFLVKWQDLSEKVVYRRFTE...,True,False,390,0,S,[99],False,390,2,2,0,1.00,0.00,"C111, C378",NaN,"{111: 'C', 378: 'C'}",2,0,0,0
1,Q96CV9,OPTN_HUMAN,577,MSHQPLSCLTEKEDSPSESTGNGPPHLAHPNLDTFTPEELLQQMKE...,True,False,577,0,E,[322],False,577,2,2,0,1.00,0.00,"K106, C239",NaN,"{106: 'K', 239: 'C'}",1,0,1,0
2,Q86YV5,PRAG1_HUMAN,1406,MHQTLCLNPESLKMSACSDFVEHIWKPGSCKNCFCLRSDHQLVAGP...,True,False,1402,-4,"S, G, A, S, S, P, F, V, P, H, L, E, S, D, Y, C...","[351, 352, 353, 354, 355, 356, 357, 358, 359, ...",False,1406,3,2,1,0.67,0.33,"C155, C221",C444,"{155: 'C', 221: 'C', 444: 'C'}",2,0,0,0
3,Q9H307,PININ_HUMAN,717,MAVAVRTLQEQLEKAKESLKNVDENIRKLTGRDPNDVRPIQARLLA...,True,False,717,0,T,[441],False,717,2,2,0,1.00,0.00,"C249, C439",NaN,"{249: 'C', 439: 'C'}",2,0,0,0
4,Q9NPH0,PPA6_HUMAN,428,MITGVFSMRLWTPVGVLTSLAYCLHQRRVALAELQEADGQCPVDRS...,True,False,428,0,M,[316],False,428,2,2,0,1.00,0.00,"C267, K291",NaN,"{267: 'C', 291: 'K'}",1,0,1,0
5,Q15283,RASA2_HUMAN,850,MAAAAPAAAAASSEAPAASATAEPEAGDQDSREVRVLQSLRGKICE...,True,False,849,-1,"S, K, D, A, I, Y, T, I, P, V, K, N, I, L, A, V...","[646, 647, 648, 649, 650, 651, 652, 653, 654, ...",False,850,1,1,0,1.00,0.00,C314,NaN,{314: 'C'},1,0,0,0
6,Q9UKL0,RCOR1_HUMAN,485,MPAMVEKGPEVSGKRRGRNNAAASASAAAASAAASAACASPAATAA...,True,False,482,-3,"P, A, M, V, E, K, G, P, E, V, S, G, K, R, G, R...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",False,485,2,0,2,0.00,1.00,NaN,"C35, K418","{35: 'C', 418: 'K'}",0,0,0,0
7,P20338,RAB4A_HUMAN,218,MSQTAMSETYDFLFKFLVIGNAGTGKSCLLHQFIEKKFKDDSNHTI...,True,False,213,-5,"Q, A, M, S, E, T, Y, D, F, L, F, K, F, L, V, I...","[3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...",False,218,1,0,1,0.00,1.00,NaN,K58,{58: 'K'},0,0,0,0
8,Q6WKZ4,RFIP1_HUMAN,1283,MSLMVSAGRGLGAVWSPTHVQVTVLQARGLRAKGPGGTSDAYAVIQ...,True,False,1283,0,"Q, A","[622, 651]",False,1283,1,1,0,1.00,0.00,C318,NaN,{318: 'C'},1,0,0,0
9,Q16822,PCKGM_HUMAN,640,MAALYRPGLRLNWHGLSPLGWPSCRSIQTLRVLSGDLGQLPTGIRD...,True,False,640,0,P,[121],False,640,4,4,0,1.00,0.00,"C92, C210, C306, C431",NaN,"{92: 'C', 210: 'C', 306: 'C', 431: 'C'}",4,0,0,0


In [18]:
# adding column to 45 file if row ID is apart of 13 IDs that were dropped for final 2018 set of 3963 IDs
dropped13 = ["Q9UKL0","P20338","Q99996","Q9Y2K2","Q66PJ3","Q5VZ89","Q03252","Q96PV6","P49750","Q99873","P06753","Q86YV5","Q8WUH1"]

mer13 = addcolumnconditional(dropped13, mer, 'ID', 'dropped13')
mer13

,ID,entryName,Length,proSequence,IDin4084list,identical_seq_x,length_2012_fasta,len12_minus_len18,difference_18vs12,posdiff_ref_shorter_pro,identical_seq_y,len_uniprot18,total_targets,found_count,missed_count,correct_frac,missed_frac,positions_found,positions_notfound,pos_dict,found_C_count,missed_C_count,found_K_count,missed_K_count,dropped13
0,P14598,NCF1_HUMAN,390,MGDTFIRHIALLGFEKRFVPSQHYVYMFLVKWQDLSEKVVYRRFTE...,True,False,390,0,S,[99],False,390,2,2,0,1.00,0.00,"C111, C378",NaN,"{111: 'C', 378: 'C'}",2,0,0,0,False
1,Q96CV9,OPTN_HUMAN,577,MSHQPLSCLTEKEDSPSESTGNGPPHLAHPNLDTFTPEELLQQMKE...,True,False,577,0,E,[322],False,577,2,2,0,1.00,0.00,"K106, C239",NaN,"{106: 'K', 239: 'C'}",1,0,1,0,False
2,Q86YV5,PRAG1_HUMAN,1406,MHQTLCLNPESLKMSACSDFVEHIWKPGSCKNCFCLRSDHQLVAGP...,True,False,1402,-4,"S, G, A, S, S, P, F, V, P, H, L, E, S, D, Y, C...","[351, 352, 353, 354, 355, 356, 357, 358, 359, ...",False,1406,3,2,1,0.67,0.33,"C155, C221",C444,"{155: 'C', 221: 'C', 444: 'C'}",2,0,0,0,True
3,Q9H307,PININ_HUMAN,717,MAVAVRTLQEQLEKAKESLKNVDENIRKLTGRDPNDVRPIQARLLA...,True,False,717,0,T,[441],False,717,2,2,0,1.00,0.00,"C249, C439",NaN,"{249: 'C', 439: 'C'}",2,0,0,0,False
4,Q9NPH0,PPA6_HUMAN,428,MITGVFSMRLWTPVGVLTSLAYCLHQRRVALAELQEADGQCPVDRS...,True,False,428,0,M,[316],False,428,2,2,0,1.00,0.00,"C267, K291",NaN,"{267: 'C', 291: 'K'}",1,0,1,0,False
5,Q15283,RASA2_HUMAN,850,MAAAAPAAAAASSEAPAASATAEPEAGDQDSREVRVLQSLRGKICE...,True,False,849,-1,"S, K, D, A, I, Y, T, I, P, V, K, N, I, L, A, V...","[646, 647, 648, 649, 650, 651, 652, 653, 654, ...",False,850,1,1,0,1.00,0.00,C314,NaN,{314: 'C'},1,0,0,0,False
6,Q9UKL0,RCOR1_HUMAN,485,MPAMVEKGPEVSGKRRGRNNAAASASAAAASAAASAACASPAATAA...,True,False,482,-3,"P, A, M, V, E, K, G, P, E, V, S, G, K, R, G, R...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",False,485,2,0,2,0.00,1.00,NaN,"C35, K418","{35: 'C', 418: 'K'}",0,0,0,0,True
7,P20338,RAB4A_HUMAN,218,MSQTAMSETYDFLFKFLVIGNAGTGKSCLLHQFIEKKFKDDSNHTI...,True,False,213,-5,"Q, A, M, S, E, T, Y, D, F, L, F, K, F, L, V, I...","[3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...",False,218,1,0,1,0.00,1.00,NaN,K58,{58: 'K'},0,0,0,0,True
8,Q6WKZ4,RFIP1_HUMAN,1283,MSLMVSAGRGLGAVWSPTHVQVTVLQARGLRAKGPGGTSDAYAVIQ...,True,False,1283,0,"Q, A","[622, 651]",False,1283,1,1,0,1.00,0.00,C318,NaN,{318: 'C'},1,0,0,0,False
9,Q16822,PCKGM_HUMAN,640,MAALYRPGLRLNWHGLSPLGWPSCRSIQTLRVLSGDLGQLPTGIRD...,True,False,640,0,P,[121],False,640,4,4,0,1.00,0.00,"C92, C210, C306, C431",NaN,"{92: 'C', 210: 'C', 306: 'C', 431: 'C'}",4,0,0,0,False


In [19]:
mer.to_csv("UKB_2018vs2012_ALL_FALSE45_with_annotations_doubleCHECK.csv", index=False)

In [20]:
# P82921 C49 detected and C50 was origianally R50 (natural variant example for why sequences not identical)
seq18 = 'MAKHLKFIARTVMVQEGNVESAYRTLNRILTMDGLIEDIKHRRYYEKPCCRRQRESYERCRRIYNMEMARKINFLMRKNRADPWQGC'
seq12 = 'MAKHLKFIARTVMVQEGNVESAYRTLNRILTMDGLIEDIKHRRYYEKPCRRRQRESYERCRRIYNMEMARKINFLMRKNRADPWQGC'

In [21]:
len('MAKHLKFIARTVMVQEGNVESAYRTLNRILTMDGLIEDIKHRRYYEKPC')

49

**keeping C49 P82921 because C50 is natural variant and Keri said while MS software would be uncertain about this if it were peptide used for mapping, the 2012 sequence of same length has no uncertainty so we can include C49 for future analysis**

# Reason why im dropping K62 Q8WUH1
- from the positions (1 based) that diff between 2018 and 2012 sequence:
    * K62 is most likely not the same K detected because all surrounding positions are different 
  
```
ID	entryName	Length	length_2012_fasta	len12_minus_len18	difference_18vs12	positions_found	posdiff_ref_shorter_pro	pos_dict

Q8WUH1	CHUR_HUMAN	139	112	-27	R, Q, P, Y, L, S, S, R, E, V, S, S, S, R, K, R, W, R, T, F, P, V, D, C, V, A, M, C, G, D, C, V, E, E, Y, P, N, R, G, N, T, C, L, E, N, G, S, F, L, L, N, F, T, G, C, A, V, C, S, R, D, F, M, L, T, N, K, S, L, K, E, E, D, G, E, I, V, T, D, H, K, N, H, H, V, I, A, R, H, E, Y, T, F, S, I, M, D, E, F, Q, E, Y, T	K62	[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 85, 86, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]	{62: 'K'}

```

file with highlight notes: UKB_DOUBLE_CHECK_CKpos_forFALSEidentical_sequences_33.xlsx

# Pmap_ACCOUNTING.py 
- script to quickly update ID/pos accounting given new final set of IDs (3963 final)

In [76]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CYS_LYS_2012_to2018UKBpositions/REFERENCE_2018_mappedCK/SUBSETS/")
print(os.listdir())

['UKBIDs_2018_3963.csv', 'UKBref2.0_2018_T3931_F32.csv', '.DS_Store', 'Lys2018_reactive_4425.csv', 'Lys2018_pantarget_8115.csv', 'CLMH_description_table_2018.csv', 'KLMH_description_table_2018.csv', 'Lys2018_ever_labeled_9046.csv', 'CYSLYS2018_everlabeled_gene_valueCounts_3964.csv', 'Cys2018_pantarget_5962.csv', 'ALLReact2018_grouped_entry_threshold_2751.csv', 'Cys2018_ever_labeled_6234.csv', 'UKB2018_reference_proteome_mappedCKpositions_3963.csv', 'dictOfPositions_allCysLys_3963.csv', 'CYS2018_everlabeled_gene_valueCounts_2915.csv', 'AllReact2018_5856.csv', 'Cys2018_reactive_1431.csv', 'LYS2018_everlabeled_gene_valueCounts_2624.csv']


In [77]:
from collections import Counter

def addcolumnconditionalDrop(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:, newcol] = mendel
    df.drop(df[df[newcol] == "False"].index, inplace = True)
    df.reset_index(inplace=True, drop=True)
    print(df.shape)
    return df

# using ref ID list to filter out subset/ensembl files for row with matching ID
def drop_IDs():
    IDdf = pd.read_csv("UKBIDs_2018_3963.csv")
    idlist = list(IDdf.ID)
    print("total IDs to be searched: ", len(idlist))
    # list of files to be cleaned
    oldfiles = ["Lys2018_reactive_4425.csv","Lys2018_pantarget_8115.csv","LYS2018_everlabeled_gene_valueCounts_2624.csv","Lys2018_ever_labeled_9046.csv","CYSLYS2018_everlabeled_gene_valueCounts_3964.csv","Cys2018_reactive_1431.csv","Cys2018_pantarget_5962.csv","CYS2018_everlabeled_gene_valueCounts_2915.csv","Cys2018_ever_labeled_6234.csv","ALLReact2018_grouped_entry_threshold_2751.csv","AllReact2018_5856.csv"]

    newfiles = ["Lys2018_reactive_4425.csv","Lys2018_pantarget_8115.csv","LYS2018_everlabeled_gene_valueCounts_2624.csv","Lys2018_ever_labeled_9046.csv","CYSLYS2018_everlabeled_gene_valueCounts_3964.csv","Cys2018_reactive_1431.csv","Cys2018_pantarget_5962.csv","CYS2018_everlabeled_gene_valueCounts_2915.csv","Cys2018_ever_labeled_6234.csv","ALLReact2018_grouped_entry_threshold_2751.csv","AllReact2018_5856.csv"]

    for i, d in enumerate(oldfiles):
        df = pd.read_csv(d)
        dropUnnamed(df)
        print("oldfile up: ", d)
        df2 = addcolumnconditionalDrop(idlist, df, 'ID', 'final_refID')
        outname = newfiles[i]
        df2.to_csv(outname, index=False)
drop_IDs()

total IDs to be searched:  3963
oldfile up:  Lys2018_reactive_4425.csv
(4425, 7)
oldfile up:  Lys2018_pantarget_8115.csv
(8111, 6)
oldfile up:  LYS2018_everlabeled_gene_valueCounts_2624.csv
(2623, 3)
oldfile up:  Lys2018_ever_labeled_9046.csv
(9042, 6)
oldfile up:  CYSLYS2018_everlabeled_gene_valueCounts_3964.csv
(3963, 3)
oldfile up:  Cys2018_reactive_1431.csv
(1431, 7)
oldfile up:  Cys2018_pantarget_5962.csv
(5962, 6)
oldfile up:  CYS2018_everlabeled_gene_valueCounts_2915.csv
(2915, 3)
oldfile up:  Cys2018_ever_labeled_6234.csv
(6234, 6)
oldfile up:  ALLReact2018_grouped_entry_threshold_2751.csv
(2751, 4)
oldfile up:  AllReact2018_5856.csv
(5856, 6)


# strange that lys panTarget file had 8115 then changed to 8111, should only have lost 2...maybe just a typo?

In [80]:
def colvalues(df, col):
    countdf = Counter(list(df[col]))
    countdic = dict(countdf)
    print(countdic)

def update_accounting():
    # UPDATED EVER LABELED FILES MANUALLY BY CHECKING NEW LINE COUNTS
    reactset = ["Lys2018_reactive_4425.csv","Cys2018_reactive_1431.csv","AllReact2018_5856.csv"]
    panset = ["Lys2018_pantarget_8111.csv","Cys2018_pantarget_5962.csv"]

    for i, d in enumerate(reactset):
        print(d)
        df = pd.read_csv(d)
        colvalues(df, 'Threshold')
        print()
        if d == "Lys2018_reactive_4425.csv":
            klow = df[df["Threshold"] == "Low"].describe()
            klow.columns = ['K_LOW']
            kmed = df[df["Threshold"] == "Medium"].describe()
            kmed.columns = ['K_MED']
            khigh=df[df["Threshold"] == "High"].describe()
            khigh.columns=['K_HIGH']
            klm = klow.join(kmed)
            klmh = klm.join(khigh)
            print(klmh)
            klmh.to_csv("KLMH_description_table_2018.csv")

        if d == "Cys2018_reactive_1431.csv":
            clow=df[df["Threshold"] == "Low"].describe()
            clow.columns=['C_LOW']
            cmed = df[df["Threshold"] == "Medium"].describe()
            cmed.columns=['C_MED']
            chigh=df[df["Threshold"] == "High"].describe()
            chigh.columns=['C_HIGH']
            CLM = clow.join(cmed)
            CLMH = CLM.join(chigh)
            print(CLMH)
            CLMH.to_csv("CLMH_description_table_2018.csv")

    for i, p in enumerate(panset):
        dfp = pd.read_csv(p)
        print(p)
        checkColumnValues(dfp, "labelType")
update_accounting()

Lys2018_reactive_4425.csv
{'High': 303, 'Medium': 666, 'Low': 3456}

             K_LOW       K_MED      K_HIGH
count  3456.000000  666.000000  303.000000
mean      9.334259    3.518769    1.240264
std       1.341434    0.881246    0.494368
min       5.100000    2.100000    0.100000
25%       9.900000    2.700000    0.900000
50%      10.000000    3.500000    1.300000
75%      10.000000    4.300000    1.700000
max      10.000000    5.000000    2.000000
Cys2018_reactive_1431.csv
{'High': 97, 'Medium': 422, 'Low': 912}

            C_LOW       C_MED     C_HIGH
count  912.000000  422.000000  97.000000
mean     7.417007    3.851872   1.336804
std      3.056919    0.831851   0.340152
min      5.010000    2.010000   0.740000
25%      5.737500    3.192500   1.060000
50%      6.460000    4.015000   1.320000
75%      7.870000    4.572500   1.640000
max     20.000000    5.000000   1.960000
AllReact2018_5856.csv
{'High': 400, 'Medium': 1088, 'Low': 4368}

Lys2018_pantarget_8111.csv
     labelType 

In [84]:
# =====DOUBLE CHECK ALL POSITIONS IN EVER LABELED FILES ARE UNIQUE
def count_unique_IDs(df, col):
    idlen = df[col]
    idset = set(df[col])
    print("ID column len: ", len(idlen))
    print("ID column set: ", len(idset))

def check_everlabeled():
    files = ["Lys2018_ever_labeled_9042.csv","Lys2018_reactive_4425.csv","Lys2018_pantarget_8111.csv","Cys2018_reactive_1431.csv","Cys2018_pantarget_5962.csv","Cys2018_ever_labeled_6234.csv","AllReact2018_5856.csv"]
    for d in files:
        print(d)
        df = pd.read_csv(d)
        print("shape: ", df.shape)
        count_unique_IDs(df, 'pos_ID')
        print()
check_everlabeled()

Lys2018_ever_labeled_9042.csv
shape:  (9042, 6)
ID column len:  9042
ID column set:  9042

Lys2018_reactive_4425.csv
shape:  (4425, 7)
ID column len:  4425
ID column set:  4425

Lys2018_pantarget_8111.csv
shape:  (8111, 6)
ID column len:  8111
ID column set:  8111

Cys2018_reactive_1431.csv
shape:  (1431, 7)
ID column len:  1431
ID column set:  1431

Cys2018_pantarget_5962.csv
shape:  (5962, 6)
ID column len:  5962
ID column set:  5962

Cys2018_ever_labeled_6234.csv
shape:  (6234, 6)
ID column len:  6234
ID column set:  6234

AllReact2018_5856.csv
shape:  (5856, 6)
ID column len:  5856
ID column set:  5856



In [87]:
def check_everlabeled_ID():
    files = ["AllPanTarget_CK_14073.csv","Lys2018_ever_labeled_9042.csv","Lys2018_reactive_4425.csv","Lys2018_pantarget_8111.csv","Cys2018_reactive_1431.csv","Cys2018_pantarget_5962.csv","Cys2018_ever_labeled_6234.csv","AllReact2018_5856.csv"]
    for d in files:
        print(d)
        df = pd.read_csv(d)
        print("shape: ", df.shape)
        count_unique_IDs(df, 'ID')
        print()
check_everlabeled_ID()

AllPanTarget_CK_14073.csv
shape:  (14073, 6)
ID column len:  14073
ID column set:  3702

Lys2018_ever_labeled_9042.csv
shape:  (9042, 6)
ID column len:  9042
ID column set:  2623

Lys2018_reactive_4425.csv
shape:  (4425, 7)
ID column len:  4425
ID column set:  1545

Lys2018_pantarget_8111.csv
shape:  (8111, 6)
ID column len:  8111
ID column set:  2369

Cys2018_reactive_1431.csv
shape:  (1431, 7)
ID column len:  1431
ID column set:  984

Cys2018_pantarget_5962.csv
shape:  (5962, 6)
ID column len:  5962
ID column set:  2788

Cys2018_ever_labeled_6234.csv
shape:  (6234, 6)
ID column len:  6234
ID column set:  2915

AllReact2018_5856.csv
shape:  (5856, 6)
ID column len:  5856
ID column set:  2078

